# Clasificador basado en transformers

[Beto ajustado para la predicción de emojis del dataset](https://huggingface.co/ccarvajal/beto-emoji)

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [2]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [3]:
MODEL = f"ccarvajal/beto-emoji"
folder = MODEL.replace('ccarvajal','modelos')

try:
    tokenizer = AutoTokenizer.from_pretrained(folder)
except ValueError:
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    tokenizer.save_pretrained(folder)

Downloading: 100%|██████████| 564/564 [00:00<00:00, 320kB/s]
Downloading: 100%|██████████| 242k/242k [00:01<00:00, 172kB/s]  
Downloading: 100%|██████████| 718k/718k [00:02<00:00, 332kB/s] 
Downloading: 100%|██████████| 112/112 [00:00<00:00, 44.2kB/s]


In [4]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/camilocarvajalreyes/beto-emoji/main/es_mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

In [5]:
try:
    model = AutoModelForSequenceClassification.from_pretrained(folder)
except OSError:
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)
    model.save_pretrained(folder)

Downloading: 100%|██████████| 1.53k/1.53k [00:00<00:00, 676kB/s]
Downloading: 100%|██████████| 419M/419M [00:30<00:00, 14.3MB/s] 


Palabras rellenar

In [6]:
def eval_text(text):
    # retorna el indice del emoji con mas probabilidad y los scores
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return np.argmax(scores), scores

In [7]:
def rank_emojis_text(text):
    # imprime los emojis ordenados por probabilidad
    _, scores = eval_text(text)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    for i in range(scores.shape[0]):
        l = labels[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [10]:
ejemplo = "que viva españa"
label, _ = eval_text(ejemplo)
print(ejemplo+'\nemoji escogido = {} ({})'.format(labels[label],label))

que viva españa
emoji escogido = 🇪🇸 (9)


In [11]:
rank_emojis_text(ejemplo)

1) 🇪🇸 0.2508
2) 😍 0.238
3) 👌 0.2225
4) 😂 0.0806
5) ❤ 0.0489
6) 😁 0.0415
7) 😜 0.0232
8) 😎 0.0229
9) 😊 0.0156
10) 😉 0.0119
11) 💜 0.0079
12) 💕 0.0077
13) 💪 0.0066
14) 💘 0.0054
15) 💙 0.0052
16) 💞 0.005
17) 😘 0.0034
18) 🎶 0.0022
19) ✨ 0.0007


Iteramos sobre el test set

In [12]:
import pickle

path =  "../../Data/test/df_es_test.pickle"
df_es_test = pickle.load(open(path, "rb"))

In [13]:
%%time
y_pred = []

for texto in df_es_test['text']:
    label, _ = eval_text(texto)
    y_pred.append(label)

CPU times: user 36min 59s, sys: 14.1 s, total: 37min 13s
Wall time: 6min 12s


In [14]:
from sklearn.metrics import classification_report
print(classification_report(df_es_test["label"].astype(int), y_pred, target_names=labels))

              precision    recall  f1-score   support

           ❤       0.39      0.43      0.41      2141
           😍       0.29      0.39      0.33      1408
           😂       0.51      0.51      0.51      1499
           💕       0.09      0.05      0.06       352
           😊       0.12      0.23      0.16       514
           😘       0.24      0.23      0.24       397
           💪       0.37      0.43      0.40       307
           😉       0.15      0.17      0.16       453
           👌       0.09      0.16      0.11       180
          🇪🇸       0.46      0.46      0.46       424
           😎       0.12      0.11      0.11       339
           💙       0.36      0.02      0.04       413
           💜       0.00      0.00      0.00       235
           😜       0.04      0.02      0.02       274
           💞       0.00      0.00      0.00        93
           ✨       0.26      0.12      0.17       416
           🎶       0.25      0.24      0.24       212
           💘       0.00    

/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/camilo/miniconda3/envs/datamining_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [18]:
from sklearn.metrics import f1_score

f1_score(df_es_test["label"].astype(int), y_pred, average='macro')

0.18153243138645886

Guardamos los resultados en pickle (la lista)

In [15]:
import pickle

with open('resultados_test/beto-emoji.pickle', 'wb') as handle:
    pickle.dump(y_pred, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
with open('resultados_test/beto-emoji.pickle', 'rb') as handle:
    preds = pickle.load(handle)

Para volver a cargar la lista

In [17]:
preds[:10]  # primeros 10 labels

[4, 7, 2, 0, 5, 0, 2, 15, 0, 8]